In [31]:
pip install neptune-client

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, make_scorer, f1_score, precision_score, recall_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold


from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
import matplotlib.pyplot as plt


from sklearn.metrics import plot_confusion_matrix
#import scikitplot as skplt
from scikitplot.estimators import plot_learning_curve



In [34]:
raw_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Train_raw.csv')
raw_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Test_raw.csv')

Adasyn_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Adasyn_train.csv')
Adasyn_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Adasyn_test.csv')

Nearmiss_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Nearmiss_train.csv')
Nearmiss_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Nearmiss_test.csv')

ROS_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\ROS_train.csv')
ROS_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\ROS_test.csv')

RUS_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\RUS_train.csv')
RUS_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\RUS_test.csv')

SMOTETomek_train = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\SMOTETomek_train.csv')
SMOTETomek_test = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\SMOTETomek_test.csv')

validation = pd.read_csv(r'E:\dataworkshop\noshows\spotkanie18\dane\Validation.csv')

In [35]:
validation

,PatientId,Gender,Age,Neighbourhood_0,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,...,DayAppointmentDate_3,DayAppointmentDate_4,DayAppointmentDate_5,HourOfDayBooking_0,DaysToAppointment,AppointmentWeekDay_0,AppointmentWeekDay_1,AppointmentWeekDay_2,AppointmentWeekDay_3,NoShow
0,342478778,1,0,0,0,1,1,1,0,1,...,1,0,0,1,14,0,0,1,0,1
1,19715663351925,0,78,0,0,1,1,0,0,1,...,1,1,1,1,0,0,0,1,1,0
2,919292581644233,1,4,0,0,0,1,0,1,1,...,1,0,1,1,0,0,0,1,1,0
3,95687334995341,0,69,0,0,0,1,1,1,1,...,1,0,0,1,0,0,1,0,1,0
4,38863974611654,1,66,0,0,1,1,1,1,1,...,0,0,1,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11048,5955176526376,1,63,0,0,0,1,1,0,1,...,0,1,0,1,6,0,1,0,0,0
11049,796386324155451,1,44,0,0,0,1,0,0,1,...,1,0,1,1,1,0,0,0,1,0
11050,59689766796,0,6,0,0,0,0,1,0,0,...,1,1,1,1,1,0,1,0,1,0
11051,1994573567149,0,77,0,0,1,0,1,0,0,...,1,1,1,1,0,0,1,0,1,0


In [36]:
import neptune

API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYmM4OTBkYTAtY2JlYi00MGM4LWIzZTUtYzg1MTJkMzJkYjk3In0=' #@param {type:"string"}

PROJEKT_NAME = 'DataWorkshop-Foundation/Predicting-cancellation-of-visit' 

neptune.init(
   api_token=API_TOKEN,
   project_qualified_name=PROJEKT_NAME
)

Project(DataWorkshop-Foundation/Predicting-cancellation-of-visit)

In [49]:
def get_feats(df):
    feats = df.columns 
    return feats[ (feats != 'NoShow')]

def get_X(df):
    return df[ get_feats(df) ]

def get_y(df, target_var='NoShow'):
    return df[target_var]

In [50]:
get_X(validation)

,PatientId,Gender,Age,Neighbourhood_0,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,...,DayAppointmentDate_2,DayAppointmentDate_3,DayAppointmentDate_4,DayAppointmentDate_5,HourOfDayBooking_0,DaysToAppointment,AppointmentWeekDay_0,AppointmentWeekDay_1,AppointmentWeekDay_2,AppointmentWeekDay_3
0,342478778,1,0,0,0,1,1,1,0,1,...,0,1,0,0,1,14,0,0,1,0
1,19715663351925,0,78,0,0,1,1,0,0,1,...,0,1,1,1,1,0,0,0,1,1
2,919292581644233,1,4,0,0,0,1,0,1,1,...,1,1,0,1,1,0,0,0,1,1
3,95687334995341,0,69,0,0,0,1,1,1,1,...,1,1,0,0,1,0,0,1,0,1
4,38863974611654,1,66,0,0,1,1,1,1,1,...,1,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11048,5955176526376,1,63,0,0,0,1,1,0,1,...,0,0,1,0,1,6,0,1,0,0
11049,796386324155451,1,44,0,0,0,1,0,0,1,...,1,1,0,1,1,1,0,0,0,1
11050,59689766796,0,6,0,0,0,0,1,0,0,...,1,1,1,1,1,1,0,1,0,1
11051,1994573567149,0,77,0,0,1,0,1,0,0,...,0,1,1,1,1,0,0,1,0,1


In [51]:
def train_model(model, X, y):
    model.fit(X, y)
    return model

In [52]:
def predict_model(model, X):
    y_pred = model.predict(X)
    return y_pred

In [53]:
def score_model(y_test, y_pred, scoring):
    score = scoring(y_test, y_pred)
    return score

In [54]:
def train_and_predict(model, X, y, scoring):
    scores = cross_val_score(model, X, y, cv = 7, scoring = scoring)
    return np.mean(scores), np.std(scores)

In [55]:
def split_data(train, test):
    feats = train.columns[:-1]
    
    X_train = train[feats]
    X_test = test[feats]
    
    y_train = train['NoShow']
    y_test = test["NoShow"]
    return X_train, y_train, X_test, y_test

In [56]:
metrics = {
    'accuracy': accuracy_score,
    'recall': recall_score,
    'precision': precision_score,
    'f1_score': f1_score,
    'roc_auc': roc_auc_score,
}


In [57]:
validation

,PatientId,Gender,Age,Neighbourhood_0,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,...,DayAppointmentDate_3,DayAppointmentDate_4,DayAppointmentDate_5,HourOfDayBooking_0,DaysToAppointment,AppointmentWeekDay_0,AppointmentWeekDay_1,AppointmentWeekDay_2,AppointmentWeekDay_3,NoShow
0,342478778,1,0,0,0,1,1,1,0,1,...,1,0,0,1,14,0,0,1,0,1
1,19715663351925,0,78,0,0,1,1,0,0,1,...,1,1,1,1,0,0,0,1,1,0
2,919292581644233,1,4,0,0,0,1,0,1,1,...,1,0,1,1,0,0,0,1,1,0
3,95687334995341,0,69,0,0,0,1,1,1,1,...,1,0,0,1,0,0,1,0,1,0
4,38863974611654,1,66,0,0,1,1,1,1,1,...,0,0,1,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11048,5955176526376,1,63,0,0,0,1,1,0,1,...,0,1,0,1,6,0,1,0,0,0
11049,796386324155451,1,44,0,0,0,1,0,0,1,...,1,0,1,1,1,0,0,0,1,0
11050,59689766796,0,6,0,0,0,0,1,0,0,...,1,1,1,1,1,0,1,0,1,0
11051,1994573567149,0,77,0,0,1,0,1,0,0,...,1,1,1,1,0,0,1,0,1,0


In [ ]:
def run_model(model, train, test, val, name, tags):
    X_train, y_train, X_test, y_test = split_data(train, test)
    X_val = get_X(val)
    y_val = get_y(val)

    neptune.create_experiment(
        name=name
    )
    
    model = train_model(model, X_train, y_train)
    y_pred = predict_model(model, X_test)
    val_pred = predict_model(model, X_val)
    
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Test")
    print(y_pred,'\n')
    print('accuracy: ', accuracy)
    print('recall: ',recall)
    print('precision: ', precision)
    print('f1: ', f1)
    print('ROC AUC: ', roc,'\n')
    print(classification_report(y_test, y_pred))


    accuracy_val = accuracy_score(y_val, val_pred)
    recall_val = recall_score(y_val, val_pred)
    precision_val = precision_score(y_val, val_pred)
    f1_val = f1_score(y_val, val_pred)
    roc_val = roc_auc_score(y_val, val_pred)
    print('------------------------------------------------------------')
    print("Validation")
    print(val_pred,'\n')
    print('accuracy: ', accuracy_val)
    print('recall: ',recall_val)
    print('precision: ', precision_val)
    print('f1: ', f1_val)
    print('ROC AUC: ', roc_val,'\n')
    print(classification_report(y_val, val_pred))
    
    neptune.log_metric('accuracy', accuracy_val)
    neptune.log_metric('recall', recall_val)
    neptune.log_metric('precision', precision_val)
    neptune.log_metric('f1', f1_val)
    neptune.log_metric('roc_auc', roc_val)
    neptune.append_tags(tags)
    neptune.append_tags(name)
    neptune.append_tags("Validation")
    neptune.append_tags(["Magdalena Cebula"])
    
    plot_confusion_matrix(model, X_val, y_val, cmap='Blues');    
    plt.show()
    plot_learning_curve(model, X_val, y_val, scoring='f1');    
    plt.show()
    neptune.stop()

In [60]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

Regresja logistyczna

RAW

In [61]:
clf = LogisticRegression(C= 2.592687488204789, l1_ratio= 0.36824153984054797, penalty= 'l1', solver= 'liblinear', random_state=0)

In [ ]:
run_model(clf, raw_train, raw_test, validation, "Logistic Regression", ['Logistic Regression', 'Raw'])

             PatientId  Gender  Age  Neighbourhood_0  Neighbourhood_1  \
0            342478778       1    0                0                0   
1       19715663351925       0   78                0                0   
2      919292581644233       1    4                0                0   
3       95687334995341       0   69                0                0   
4       38863974611654       1   66                0                0   
...                ...     ...  ...              ...              ...   
11048    5955176526376       1   63                0                0   
11049  796386324155451       1   44                0                0   
11050      59689766796       0    6                0                0   
11051    1994573567149       0   77                0                0   
11052    1347154834816       0   23                0                0   

       Neighbourhood_2  Neighbourhood_3  Neighbourhood_4  Neighbourhood_5  \
0                    1                1       

https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-415


D:\Users\Magda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn("l1_ratio parameter is only used when penalty is "


RUS

In [ ]:
run_model(clf, RUS_train, RUS_test, validation, "Logistic Regression", ['Logistic Regression', 'RUS'])

ROS

In [ ]:
run_model(clf, ROS_train, ROS_test, validation, "Logistic Regression", ['Logistic Regression', 'ROS'])

SMOTETomek

In [ ]:
run_model(clf, SMOTETomek_train, SMOTETomek_test, validation, "Logistic Regression", ['Logistic Regression', 'SMOTETomek'])

Adasyn

In [ ]:
run_model(clf, Adasyn_train, Adasyn_test, validation, "Logistic Regression", ['Logistic Regression', 'Adasyn'])

Nearmiss

In [ ]:
run_model(clf, Nearmiss_train, Nearmiss_test, validation, "Logistic Regression", ['Logistic Regression', 'Nearmiss'])

    'Dummy': DummyClassifier(strategy="most_frequent"),
    'DecissionTree' : DecisionTreeClassifier(criterion= 'gini', max_depth= 28, max_features= 17, random_state=0),
    'ExtraTrees': ExtraTreesClassifier(n_estimators=100, random_state=0),
    'RandomForest' :RandomForestClassifier(max_depth=2, random_state=0),

In [ ]:
metrics = {
    'accuracy': accuracy_score,
    'recall': recall_score,
    'precision': precision_score,
    'f1_score': f1_score,
    'roc_auc': roc_auc_score,
}

models = {
    'Dummy': DummyClassifier(strategy="most_frequent"),
    'DecissionTree' : DecisionTreeClassifier(criterion= 'gini', max_depth= 28, max_features= 17, random_state=0),
    'ExtraTrees': ExtraTreesClassifier(n_estimators=100, random_state=0),
    'RandomForest' :RandomForestClassifier(max_depth=2, random_state=0),
    'AdaBoost':AdaBoostClassifier(n_estimators=100, random_state=0),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
    'SVC': SVC(gamma='auto'),
    'KNN': KNeighborsClassifier(),
    'RadiusNeighbors': RadiusNeighborsClassifier(radius=1.0),
    'NearestCentroid': NearestCentroid(),
    'LDA': LinearDiscriminantAnalysis(n_components=2),
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
    'BernoulliNB': BernoulliNB()
}

In [ ]:
tags = ['Raw']
for model in models.items():
    print('############################################################################')
    print(model[0])
    run_model(model[1], raw_train, raw_test, validation, model[0], tags)
    print('****************************************************************************')

Wnioski:

1) zapisywanie modeli

2) walidacja od razu na najlepszym z modeli